## Correct 1D LCT Including Units and Wavelength

In [4]:
import numpy as np
import time as tm
#import math
#import os 
#import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# import scipy.ndimage
# import scipy.ndimage as ip
# from srwlib import *
from LCTlib import *

# plt.style.available
# plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12
# plt.style.use('Solarize_Light2')
hfontLarge = {'fontname':'Latin Modern Roman', 'size' : 24, 'weight' : 'bold'}
hfontMed = {'fontname':'Latin Modern Roman', 'size' : 15}